In [1]:
%matplotlib inline
import json 
import matplotlib.pyplot as plt
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import keras 

# import numpy
# import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf


from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix


from keras import backend as K

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer

# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from xgboost import plot_tree
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [2]:
#  mapps the input records to a integer array for the input
def mapping_x( inp, include_direction = False , TrimAt= 15 ):
    if include_direction:
        return np.array([ int(x["packet_length"]) * (1 if x['packet_source']=='hub' else -1)  for x in inp ][:15])
    else:
        return np.array([ int(x["packet_length"])  for x in inp ][:15])

In [3]:
def mapping_y_service(inp):
    return np.array(  list(set([x["event"] for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_service_event(inp):
    return np.array(  list(set([ "%s-%s"%( x["event"] ,x["val"] ) for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_device_service(inp):
    return np.array(  list(set([ "%s & %s"%( x["device"] ,x["event"] ) for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_full(inp):
    return np.array(  list(set([ "%s & %s & %s"%( x["device"] ,x["event"], x['val'] ) for x in inp])) if (len(inp )>0) else ["none"] )


In [4]:
def clean_data( x_data, y_data , removeempty=True, Mapping='S', include_direction=False):
    cleans = [] 
    cleans = (sorted([ int(x) for x in y_data if (removeempty and len(y_data[x]) > 0) or not removeempty  ] ))
    
    ret_x  = [x_data[ str(x)] for x in cleans]
    ret_y  = [y_data[str(x)] for x in cleans] 
    
    print( len(y_data), len(cleans) )
    
    ret_x  = [ mapping_x(x, include_direction=include_direction) for x in ret_x ] 
    ret_y_s = [ mapping_y_service(y) for y in ret_y ]
    if Mapping=='S':
        ret_y  = [ mapping_y_service(y) for y in ret_y ]
    elif Mapping=='SE':
        ret_y  = [ mapping_y_service_event(y) for y in ret_y ]
    elif Mapping=='DS':
        ret_y  = [ mapping_y_device_service(y) for y in ret_y ]
    elif Mapping=='F':
        ret_y  = [ mapping_y_full(y) for y in ret_y ]
    
#     ret_y = ret_y.tolist()
#     ret_y_s = ret_y_s.tolist()
    
#     for t in ret_y:
#         if 'no_logs' in t:
#             t.remove('no_logs')
#             t.append('unknown')
#     ret_y_s.remove('no_logs')
    
#     ret_y = np.array(ret_y)
#     ret_y_s = np.array(ret_y_s)
    
    return ret_x, ret_y, ret_y_s

In [ ]:
def is_clean_event( inp, return_clean= True  ):
    return is_clean(inp, return_clean=return_clean, to_keep=[ 'no_logs', 'lock-unlocked', 'on/off-XXX', 'raw-XXX', 'read_attr_-_raw-XXX' ] )
#     if return_clean:
#         return  'no_logs' not in inp and 'lock-unlocked' not in inp and 'on/off-XXX' not in inp and 'raw-XXX' not in inp and 'read_attr_-_raw-XXX' not in inp
#     else:
#         return  'lock-locked' in inp or 'lock-unlocked'  in inp or 'on/off-XXX' in inp or  'raw-XXX' in inp  or 'read_attr_-_raw-XXX' in inp 
     
def is_clean_service( inp, return_clean= True  ):
    return is_clean(inp, to_keep=['no_logs','unknown', 'read_attr_-_raw'], return_clean=return_clean )
    
#     if return_clean:
#         return  'no_logs' not in inp and 'unknown' not in inp and 'read_attr_-_raw' not in inp #and 'ping' not in inp 
#     else:
#         return  'no_logs' in inp or  'unknown' in inp  or 'read_attr_-_raw' in inp #or 'ping' in inp 

def is_clean(inp, to_keep=[], return_clean=True):
    ret = False 
    
    for x  in to_keep:
        if x in inp:
            ret = True
            
    if not return_clean:
        ret = not ret
        
    return ret
    

In [5]:
def pre_process_raw( x_data,y_data, dim_size = 128, zero_pad = False, normalize = False ,
                    classes=None, twoD= False, as_string=False, class_cap = 9999999, cheat=True, remove_empty=True ):
#  y data 
# """
# this functino is in charge of preprocessing the records , the sourc e json contains a lot of extra stuff, this function tailors
# the data and it fixes their lenghth
# """
    if classes is None:
        classes  = sorted(list(np.unique( np.concatenate( y_data  ))))
    else :
        classes = sorted(classes)
    y_data_categorical = []  

    to_keep_indexes = []
    class_counts= {x:0 for x in classes}
    unknown_index = -1
    for i,x in enumerate(classes):
        if 'unknown' in x:
            unknown_index = i
            break

    for pick in ['ping','ping-ping','unknown']:
        if pick in class_counts:
            class_counts[pick] = -9999999

    for i,x in enumerate(y_data):
       
        temp = np.zeros(len(classes))
        x= list(set(x))
        for y in x: 
            y=y.strip()
            if cheat and (y=='unknown' or y=='unknown-') and len(x)>1:
                    continue
            
            if y in classes:
                
                if class_counts[y] < class_cap:
                    temp[classes.index(y)] = 1
                    class_counts[y] += 1
          
        if np.sum(temp) > 0 :    
            y_data_categorical.append(temp)
            to_keep_indexes.append(i)
        elif np.sum(temp) == 0 and not remove_empty and class_cap < 10000:
            temp[unknown_index] = 1
            y_data_categorical.append(temp)
            to_keep_indexes.append(i)
            
    y_data_categorical = np.vstack(y_data_categorical)
    
#     x_data = np.array( x_data) / 1500.0
    
    x_data_temp = [] 
    
    if not zero_pad:
        if twoD:
            for i,x in enumerate( x_data):
                if i not in to_keep_indexes:
                    continue
                    
                temp = [] #list(x)
                lst = list(x)
                while dim_size**2 - len(temp )   > len(lst):
                    temp.extend(lst)

                while len(temp) < dim_size**2:
                    temp.append( 0 )

                x_data_temp.append(np.array(temp).reshape(dim_size,dim_size))


            x_data_temp = np.array( x_data_temp )
            x_data_temp=x_data_temp.reshape(x_data_temp.shape+(1,))
        else: 
            temp = [] 
            lst = list(x)
            for i,x in enumerate( x_data):
                if i not in to_keep_indexes:
                    continue
                temp = [] #list(x)
                lst = list(x)
                while dim_size - len(temp )   > len(lst):
                    temp.extend(lst)

                while len(temp) < dim_size:
                    temp.append( 0 )
                
                x_data_temp.append(np.array( temp))
            
    else :
        x_data_temp = sequence.pad_sequences(x_data, maxlen=dim_size)[to_keep_indexes]
    
    
    
    if normalize:
        x_data_temp = np.array( x_data_temp) / (np.amax( x_data_temp) + 0.000000000001)
    else :
        x_data_temp = np.array(x_data_temp)
    
    
    if as_string:
        x_data_temp = [ ' '.join(list(map(str,x))) for x in x_data_temp ]
    
    return x_data_temp ,y_data_categorical , (classes, to_keep_indexes)

# Model Evaluation Methods

In [6]:
def make_recall_shit( inp ):
    tp = inp[1][1]
    tn = inp[0][0]
    fp = inp[0][1] 
    fn = inp[1][0]
    
    acc = (tp+tn)*1.0 / ( tp+tn+fp+fn)*1.0
    recall = tp*1.0/ ( tp+fn ) *1.0
    prec = tp*1.0 / ( tp+fp )*1.0
    
#     F= 2.0*( prec* recall )/ (prec+recall)
    F= 2.0*( tp)/ (2*tp + fp + fn)
    
    return acc, recall, prec, F

def acc_match( true, pred ):
    """
    returns exact mathc accuracy
    """
 
    return (len( [ x  for x  in  [np.sum(np.abs( true[i]- pred[i] )) for i in range(len(true))] if x  == 0]))*1.0 / len(true)


# def acc_none_zero ( true, pred ):
    

def acc_match_wierd( true, pred ):
    """
    returns exact mathc accuracy
    """
    level = 6 
    switch = 11
    threeAxis=13
    accel = 0 
    status=10
    contact=5
    
    counter  = 0 
    for i in range( len (true) ):
        if np.sum(np.abs( true[i]- pred[i] ))==0 :
            counter+=1
        else : 
            t_rec = np.array(list( pred[i]))
            
            if true[i][level]==1 and true[i][switch]==1 and t_rec[level]==1 :
                t_rec[switch]=1
            
            if true[i][threeAxis]==1 and true[i][accel]==1 and t_rec[threeAxis]==1:
                t_rec[accel] =1
            
            if true[i][status]==1 and true[i][contact]==1 and t_rec[status]==1:
                t_rec[contact]=1
#             print(t_rec , true[i])    
            if np.sum(np.abs( true[i]- t_rec ))==0 :
                counter+=1   
            
             
            
    
    return counter*1.0 / len(true)


def print_info(y_test, pred , classes , confidance=0.5, print_skf1=False):
    
    counts = np.sum( y_test.astype(int) , axis=0)
    
    pred[pred>=confidance] = 1
    pred[pred<confidance] = 0
    
#     acc_wierd  =acc_match_wierd(y_test, pred)
    
    conf= multilabel_confusion_matrix( y_test , pred.astype(int), labels= range(len(classes)))
    accs = [make_recall_shit(x) for x in conf]
    print( "%30s  %8s   %8s  %8s  %8s %8s %22s"  %( "Class","Accuracy", "Recall","Precision","F Score" , "Count", "TP/TN/FP/FN"))
    print( "------------------------------------------------------------------------" )
    ret  = [] 
    ret_description = ["Class","Accuracy", "Recall","Precision","F Score" , "Count", "TP","TN","FP","FN"]
    for index in range(len(classes)):
        tp = conf[index][1][1]
        tn = conf[index][0][0]
        fp = conf[index][0][1] 
        fn = conf[index][1][0]
        item = (classes[index],
              accs[index][0],
              accs[index][1],
              accs[index][2],
              accs[index][3],
              counts[index],
                  tp ,
                tn ,
                fp ,
                fn)
        print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %5d/%5d/%5d/%5d"  %  item )
        ret.append(item)
        
    n_zeros_true = len([ x  for x  in  [np.sum(np.abs( y_test[i] )) for i in range(len(y_test))] if x  == 0]  )
    n_zeros_pred = len([ x  for x  in  [np.sum(np.abs( pred[i] )) for i in range(len(pred))] if x  == 0]  )
    
    accs = np.nan_to_num(accs)
    
    print ("------------------------------------------------------------------------")
    print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %5d/%5d/%5d/%5d"  %
             ("AVERAGES",
              np.average( accs, axis=0)[0],
              np.average( accs, axis=0)[1],
              np.average( accs, axis=0)[2],
              np.average( accs, axis=0)[3],
              len(y_test),
                  0 ,
                0,
                0 ,
                0 ))
    print ("------------------------------------------------------------------------")
    print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %5d/%5d/%5d/%5d"  %
             ("Weighted AVERAGES",
              np.average( accs, weights=counts, axis=0)[0],
              np.average( accs, weights=counts, axis=0)[1],
              np.average( accs, weights=counts, axis=0)[2],
              np.average( accs, weights=counts, axis=0)[3],
              len(y_test),
                  0 ,
                0,
                0 ,
                0 ))
    
    print ("------------------------------------------------------------------------")
    unknonw_index = [classes.index('unknown')]
#     print(unknonw_index, len(accs))
    accs_new = np.delete(accs,unknonw_index,0)
    counts_new = [x for i,x in enumerate(counts) if i not in unknonw_index]
#     print(unknonw_index, len(counts), len(counts_new),len(accs), len(accs_new))
#     print(unknonw_index, len(accs))
    print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %5d/%5d/%5d/%5d"  %
             ("known Weighted AVERAGES",
              np.average( accs_new, weights=counts_new, axis=0)[0],
              np.average( accs_new, weights=counts_new, axis=0)[1],
              np.average( accs_new, weights=counts_new, axis=0)[2],
              np.average( accs_new, weights=counts_new, axis=0)[3],
              len(y_test),
                  0 ,
                0,
                0 ,
                0 ))
    
    print ("------------------------------------------------------------------------")
    if 'ping' in classes:
        unknonw_index = [classes.index('unknown'), classes.index('ping')]
    #     print(unknonw_index, len(accs))
        accs_new = np.delete(accs,unknonw_index,0)
        counts_new = [x for i,x in enumerate(counts) if i not in unknonw_index]
    #     print(unknonw_index, len(counts), len(counts_new),len(accs), len(accs_new))
    #     print(unknonw_index, len(accs))
        print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %5d/%5d/%5d/%5d"  %
                 ("known -ping Weighted AVERAGES",
                  np.average( accs_new, weights=counts_new, axis=0)[0],
                  np.average( accs_new, weights=counts_new, axis=0)[1],
                  np.average( accs_new, weights=counts_new, axis=0)[2],
                  np.average( accs_new, weights=counts_new, axis=0)[3],
                  len(y_test),
                      0 ,
                    0,
                    0 ,
                    0 ))

    print ("------------------------------------------------------------------------")
    unknonw_index = classes.index('unknown')
#     print(unknonw_index, len(accs))
    to_delete = [i for i,x in enumerate(counts) if x == 0 ]
    accs_new = np.delete(accs,to_delete,0)
    counts_new = [x for i,x in enumerate(counts) if i not in  to_delete]
#     print(unknonw_index, len(counts), len(counts_new),len(accs), len(accs_new))
#     print(unknonw_index, len(accs))
    print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %5d/%5d/%5d/%5d"  %
             ("non zero count AVERAGES",
              np.average( accs_new, axis=0)[0],
              np.average( accs_new, axis=0)[1],
              np.average( accs_new, axis=0)[2],
              np.average( accs_new, axis=0)[3],
              len(y_test),
                  0 ,
                0,
                0 ,
                0 ))
    
    print ("------------------------------------------------------------------------")
    from sklearn.metrics import f1_score
    f1 = f1_score(y_test, pred, average='macro')
    print("Macro F1 : %f" % f1)
    f1 = f1_score(y_test, pred, average='samples')
    print("sample F1 : %f" % f1)
    f1 = f1_score(y_test, pred, average='weighted')
    print("weighted F1 : %f" % f1)

    
    if 'ping' in classes and print_skf1:
        unknonw_index = classes.index('unknown')
        ping_index = classes.index('ping')
        print ("--------------------------removed ping and unknown----------------------------------------------")
        to_remove_indexes = [i for i,x in enumerate(pred) if x[ping_index]+x[unknonw_index] > 0.5]
        new_pred   =np.array( [x for i,x  in enumerate(pred) if i not in to_remove_indexes ])
        new_y_test = np.array([x for i,x  in enumerate(y_test) if i not in to_remove_indexes ])
        f1 = f1_score(new_y_test, new_pred, average='macro')
        print("Macro F1 : %f" % f1)
        f1 = f1_score(new_y_test, new_pred, average='samples')
        print("sample F1 : %f" % f1)
        f1 = f1_score(new_y_test, new_pred, average='weighted')
        print("weighted F1 : %f" % f1)

    
    
    print ( "Exact Match ACC : %.5f " % acc_match( y_test, pred )  )
#     print ( "Wierd Exact Match ACC : %.5f" % acc_wierd)
    print ( "Total Records : %d " % len(y_test)  )
    print ( "Total ZXeros in True : %d (%.3f)%%" % (n_zeros_true ,  n_zeros_true * 1.0/ len(y_test)  ))
    print ( "Total ZXeros in Test : %d (%.3f)%%" % (n_zeros_pred ,  n_zeros_pred * 1.0/ len(y_test)  ) )
    print ('=============================================================================')
    
    return ret, ret_description
    
    
    
def make_readable_results ( inp , classes , conffidance=True):
    ret = [] 
    inp =inp.astype(int)
    for xx in range(len(inp)) :
        u = inp[xx]
        temp = []
        for j in range(len(u)) : 
            if u[j] >0:
                temp.append(classes[j])
        ret.append(temp)
    return ret


def makeReadable( model , data, gt, path , classes, x, confidance=0.5):
    #collect across multi models
    from keras.models import load_model
    x=[]
    for i in range(0,16):
        model=load_model('number'+str(i)+'.h5',custom_objects={'f1_perRow':f1_perRow,'f1_perClass':f1_perClass,'f1_loss_perRow':f1_loss_perRow,'f1_loss_perClass':f1_loss_perClass})
        x.append(model.predict(data))
    x=np.array(x)
    x=np.transpose(x)
    x=np.squeeze(x )
    print(x.shape)
        
        
    pred_temp  = x
    #pred_temp = model.predict(data)
    print_info(gt, x, classes , confidance=confidance)
    
    
def add_unknowns_back(y_pred, original_y, indexes, classes):
    unknown = [1 if classes[i] == 'unknown' else
                        0  
                       for i in range(len(classes))]
    the_y = [unknown for i in range(len(original_y))]
    for i, v in enumerate(indexes):
#         print(v)
        the_y[v] = y_pred[i]
    return np.array(the_y)

In [ ]:
## Bidirectional LSTM :D 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Flatten


import numpy as np
import os
import tensorflow as tf
from keras.models import Model

from keras.layers import Input, merge, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, Dropout, Conv2DTranspose, UpSampling2D, Lambda
from keras.layers import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization as bn
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras import regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import add
from keras.models import load_model
import numpy as np
from keras.regularizers import l2
from keras.layers import Conv1D
from keras.layers.merge import add
from keras.layers import Dense,Conv1D,Dropout,Activation,BatchNormalization,MaxPooling1D,Flatten,Masking,TimeDistributed
from keras.layers.recurrent import LSTM,GRU,SimpleRNN
from keras.models import Input,Sequential,Model
from keras.layers.merge import add
from keras.optimizers import Adam
from keras.losses import MSE,MSLE
model2 = Sequential()


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss


import tensorflow as tf

def f1_perClass(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


def f1_perRow(y_true, y_pred):

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss_perRow(y_true, y_pred):
    
#     y_true = y_true * 
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1) 
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1) 
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1) 
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1) 

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return (1 - K.mean(f1))**2

def f1_loss_perClass(y_true, y_pred):
    key = K.variable([1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.])
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = (2*p*r / (p+r+K.epsilon()))*key
    
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return (1 - K.mean(f1))**2